Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved. This source code is licensed under the license found in the LICENSE file in the root directory of this source tree.

# Video Seal - Video inference

In [1]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ..

/private/home/pfz/09-videoseal/fbresearch-new


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from videoseal.utils.display import save_vid
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, pvalue, capacity, psnr, ssim, msssim, linf
from videoseal.data.datasets import VideoDataset
from videoseal.augmentation import Identity, H264, Crop
from videoseal.modules.jnd import JND

import os
from tqdm import tqdm
import torch
import gc

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda" 
device = "cpu" 

/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from tqdm import tqdm
import torch

# seed
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)

# Directory containing videos
video_dir = "assets/videos/"
base_output_folder = "outputs"
if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

# Example usage
ckpts = {
    # "trustmark": "baseline/trustmark",
    # "model": "baseline/model",
    # "cin": "baseline/cin",
    # "mbrs": "baseline/mbrs",
    # "videoseal_0.0": 'videoseal_0.0',
    "videoseal": 'videoseal',
}

fps = 24 // 1
num_videos = None

# a timer to measure the time
timer = Timer()

# Iterate over all ckpts
for model_name, ckpt in ckpts.items():
    model = setup_model_from_checkpoint(ckpt)
    model.eval()
    model.compile()
    model.to(device)

    model.step_size = 4
    model.video_mode = "repeat"

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][:num_videos]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {model_name}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        timer.start()
        vid, mask = VideoDataset.load_full_video_decord(video_path)
        print(f"loading video {video_path} - took {timer.stop():.2f}s")

        # Watermark embedding
        timer.start()
        outputs = model.embed(vid, is_video=True, lowres_attenuation=True)
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs = vid  # b c h w
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # # save
        timer.start()
        save_vid(imgs, f"{base_output_folder}/{model_name}_{base_name}_ori.mp4", fps)
        save_vid(imgs_w, f"{base_output_folder}/{model_name}_{base_name}_wm.mp4", fps)
        save_vid(10*diff.abs(), f"{base_output_folder}/{model_name}_{base_name}_diff.mp4", fps)

        # Metrics
        metrics = {
            "psnr": psnr(imgs, imgs_w, is_video=True).mean().item(),
            # "ssim": ssim(imgs, imgs_w).mean().item(),
            # "msssim": msssim(imgs, imgs_w).mean().item(),
            # "linf": linf(imgs, imgs_w).mean().item()
        }

        # Augment video
        print(f"compressing and detecting watermarks")
        for ii in range(4):
        # for ii in range(1):
            if ii == 0:
                imgs_aug = imgs_w
                label = "Original"
            if ii == 1: 
                imgs_aug, _ = H264()(imgs_w, crf=30)
                imgs_aug, _ = Crop()(imgs_aug, size=0.75)
                label = "H264 30 + Crop 0.8"
            if ii == 2: 
                imgs_aug, _ = H264()(imgs_w, crf=40)
                label = "H264 40"
            if ii == 3: 
                imgs_aug, _ = H264()(imgs_w, crf=50)
                label = "H264 50"

            # detect
            timer.start()
            aggregate = True
            if not aggregate:
                outputs = model.detect(imgs_aug, is_video=True)
                preds = outputs["preds"]
                bit_preds = preds[:, 1:]  # b k ...
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - took {timer.stop():.2f}s")
            else:
                bit_preds = model.extract_message(imgs_aug)
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                pvalue_ = pvalue(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                capacity_ = capacity(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                metrics[f"pvalue_{label.lower().replace(' ', '_').replace('.', '')}"] = pvalue_
                metrics[f"capacity_{label.lower().replace(' ', '_').replace('.', '')}"] = capacity_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - P-Value: {pvalue_:0.2e} - Capacity: {capacity_:.3f} - took {timer.stop():.2f}s")
        print(metrics)

        del vid, outputs, imgs, imgs_w, diff

File /private/home/pfz/09-videoseal/fbresearch-new/ckpts/y_256b_img.pth exists, skipping download
Model loaded successfully from /private/home/pfz/09-videoseal/fbresearch-new/ckpts/y_256b_img.pth with message: <All keys matched successfully>


Processing Videos for videoseal:   0%|          | 0/1 [00:00<?, ?it/s]

loading video assets/videos/1.mp4 - took 24.40s
embedding watermark  - took 14.85s
compressing and detecting watermarks
{'psnr': 48.327247619628906, 'bit_accuracy_original': 1.0, 'pvalue_original': 8.636168555094445e-78, 'capacity_original': 256.0, 'bit_accuracy_h264_30_+_crop_08': 0.61328125, 'pvalue_h264_30_+_crop_08': 0.00017424190213637938, 'capacity_h264_30_+_crop_08': 9.561767578125, 'bit_accuracy_h264_40': 0.796875, 'pvalue_h264_40': 9.255391288366052e-23, 'capacity_h264_40': 69.59762573242188, 'bit_accuracy_h264_50': 0.53515625, 'pvalue_h264_50': 0.1439946432658289, 'capacity_h264_50': 0.9136962890625}


Processing Videos for videoseal: 100%|██████████| 1/1 [02:54<00:00, 174.94s/it]
